In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import einops
from src.layers.flow import get_mask, MaskedLinear

In [ ]:
x = torch.randn(32,10,2)
x_hat = torch.randn(32,10,2)

mse_loss = torch.nn.functional.mse_loss(x_hat, x) * 10 * 2
sum_loss = torch.nn.functional.mse_loss(x_hat, x, reduction='sum') / 32
print(mse_loss)
print(sum_loss)

In [ ]:
x.sum()/32

In [ ]:
x.sum(dim=tuple(range(1, x.ndim))).mean()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
res = np.load('/home/user/data2/ICML_rebuttal/savings/uncond/etth2_24_S/KoVAEorig/cond_None_dtm_True_syn.npy')
# res = np.load('/home/user/data/ICML_rebuttal/savings/uncond/etth2_24_S/KoVAE_orig/cond_None_dtm_True_syn.npy')
start = np.random.randint(0, len(res))
_ = plt.plot(res[start, :, :].squeeze().T)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('/home/user/workspaces/THU-timeseries/ETT-small/ETTh2.csv', index_col=0)
# df.values.shape
start = np.random.randint(0, len(df)-24)
_ = plt.plot(df.values[start:start+24, :])

In [ ]:
from abc import ABC, abstractmethod
from typing import List, Optional, Literal
from lightning import LightningModule
import torch
from torch.nn import functional as F

def _condition_shape_check(n_sample, condition, cond_type):
    assert n_sample >=1
    if cond_type is None:
    
        if condition.shape[0] == 1:
            condition = condition.repeat(
                n_sample, *[1 for _ in range(len(condition.shape) - 1)]
            )
        elif condition.shape[0] == n_sample:
            pass
        else:
            raise ValueError(
                "The batch size of the given condition should be the same as n_sample or just 1."
            )
 
    return condition


class BaseModel(ABC, LightningModule):
    """Base class for generative models in PyTorch Lightning"""
    ALLOW_CONDITION = ...
    
    def __init__(self, seq_len, seq_dim, condition, lr, **kwargs):
        super().__init__()
        if condition not in self.ALLOW_CONDITION:
            raise ValueError(f"Condition '{condition}' not allowed. Choose from {self.ALLOW_CONDITION}")
    
    @torch.no_grad()  # wrap with torch.no_grad()
    def sample(self, n_sample: int = 1, condition=None, **kwargs):
        """Generate samples from the generative model"""
        condition = _condition_shape_check(n_sample, condition, self.condition)
        self.eval()
        return self._sample_impl(n_sample, condition, **kwargs)

    @abstractmethod
    def _sample_impl(self, n_sample: int = 1, condition=None, **kwargs) -> torch.Tensor:
        """Actual implementation of the sampling process"""


class MyModel(BaseModel):
    """AAAAAAA

    Args:
        BaseModel (_type_): _description_

    Returns:
        _type_: _description_
    """
    ALLOW_CONDITION = [None, 'predict']
    def __init__(self, seq_len, seq_dim, condition, lr, **kwargs):
        """aaaaaaa

        Args:
            seq_len (_type_): _description_
            seq_dim (_type_): _description_
            condition (_type_): _description_
            lr (_type_): _description_
        """
        super().__init__(seq_len, seq_dim, condition, lr, **kwargs)
    
    def _sample_impl(self, n_sample = 1, condition=None, **kwargs):
        return super()._sample_impl(n_sample, condition, **kwargs)

MyModel
model = MyModel(24, 1, 'impute', 0.001)

In [ ]:
import torch

total_seq_len = 64
num_samples = 200
t = torch.linspace(0, 4 * torch.pi, total_seq_len).float()
curves = []
labels = []
for _ in range(num_samples):
    a = torch.rand(1).item() * 0.5  # Initial radius
    b = torch.rand(1).item() * 0.2  # Growth rate

    direction = torch.randint(0, 2, (1,)).item()  # 0=clockwise, 1=ccw

    r = a + b * t
    if direction == 0:
        x = r * torch.cos(t)
        y = r * torch.sin(t)
    else:
        x = -r * torch.cos(t)
        y = r * torch.sin(t)

    x += torch.randn_like(x) * 0.01
    y += torch.randn_like(y) * 0.01

    curve = torch.stack([x, y], dim=1)
    curves.append(curve)
    labels.append(direction)
data, class_cond = torch.stack(curves), torch.tensor(labels).unsqueeze(-1)
print(data.shape, class_cond.shape)

class_cond[1,...,...]

In [4]:
import torchaudio.transforms as T
import torch

n_fft = 200
hop_length = 10
data = torch.randn(32, 64, 5)
hop_length = 8
print((data.shape[1]) // hop_length + 1)
data = torch.permute(
    data, (0, 2, 1)
)  # we permute to match requirements of torchaudio.transforms.Spectrogram
spec = T.Spectrogram(n_fft=15, hop_length=hop_length, center=True, power=None).to(data.device)
transformed_data = spec(data)
transformed_data.shape

9


torch.Size([32, 5, 8, 8])

In [172]:
import numpy as np

def sine_data_generation(no, seq_len, dim, freq_scale=1):

    """Sine data generation.
  
    Args:
    
    - no: the number of samples
    - seq_len: sequence length of the time-series
    - dim: feature dimensions
    
    Returns:
    - data: generated data
    
    """
    # Initialize the output
    data = list()

    # Generate sine data
    for i in range(no):      
        # Initialize each time-series
        temp = list()
        # For each feature
        for k in range(dim):
            # Randomly drawn frequency and phase
            freq = np.random.uniform(0.05, 0.4)            
            phase = np.random.uniform(0, 1.5)
                
            # Generate sine signal based on the drawn frequency and phase
            temp_data = [np.sin(freq * j + phase) for j in range(seq_len)] 
            temp.append(temp_data)
            
        # Align row/column
        temp = np.transpose(np.asarray(temp))        
        # Normalize to [0,1]
        temp = (temp + 1)*0.5
        # Stack the generated data
        data.append(temp)
                    
    return data

In [18]:
from src.model.flow.fourierflow._fourier import DFT, reconstruct_DFT
import numpy as np
import torch

t = np.linspace(0, 6, 65)
k = int(len(t) / 2) + 1

x = np.sin(t) + np.random.randn(*t.shape) * 0.1

x = x.reshape(-1, len(t))
x = torch.from_numpy(x)
m = DFT(x.shape[1])
z, log_pz, log_jacob = m(x)
z = torch.complex(z[:, 0, :], z[:, 1, :])
print("FF implement")
print(z.shape)
print(z)

print("torch.fft.rfft implement")
x_torch_fft = torch.fft.rfft(x) / len(t)
print(x_torch_fft.shape)
x_torch_fft = x_torch_fft.flip(dims=[1]).conj().type_as(z)
print(x_torch_fft.shape)
print(x_torch_fft)
torch.testing.assert_close(x_torch_fft, z)

# x_torch_fft = np.fft.fft(x)
# x_torch_fft = np.fft.fftshift(x_torch_fft)
# print(x_torch_fft.shape)
# print(x_torch_fft)

# x_torch = torch.complex(x_numpy[:,0,:], x_numpy[:,1,:])
# print(x_torch.shape)
# x_irfft = torch.fft.irfft(x_torch, dim=1)
# x_irfft.shape

# x_numpy_r = reconstruct_DFT(x_numpy[0, :, :], component="real").detach().numpy()

# x_numpy_i = reconstruct_DFT(x_numpy[0, :, :], component="imag").detach().numpy()
# print(x_numpy_r.shape)
# print(x_numpy_i.shape)
# np.real(np.fft.ifft(np.fft.ifftshift(x_numpy_r + 1j * x_numpy_i))).shape


FF implement
torch.Size([1, 33])
tensor([[-0.0224-2.7048e-03j,  0.0062-9.1164e-03j, -0.0019+1.5549e-02j,
          0.0030+1.0849e-03j, -0.0001-4.0076e-03j, -0.0045-5.5286e-03j,
          0.0028+5.9277e-03j,  0.0050-7.4491e-05j,  0.0083-1.6239e-02j,
         -0.0066+8.7482e-03j,  0.0036+5.4848e-03j,  0.0008-1.8592e-03j,
          0.0083+1.3301e-03j,  0.0058-3.1463e-03j, -0.0087-1.2385e-02j,
         -0.0048-2.4872e-03j, -0.0013-8.0844e-04j, -0.0040-1.5267e-02j,
         -0.0082+1.9869e-03j, -0.0046-7.0200e-03j,  0.0139-3.2835e-03j,
          0.0007+1.4498e-02j,  0.0053-4.7779e-03j, -0.0033-1.0216e-02j,
          0.0023+4.6939e-03j,  0.0018+1.6374e-02j, -0.0067+1.1505e-02j,
          0.0127+4.6092e-03j, -0.0029+1.9534e-02j, -0.0081+1.0713e-04j,
         -0.0078+3.4772e-02j, -0.0490+4.9984e-01j, -0.0002+0.0000e+00j]])
torch.fft.rfft implement
torch.Size([1, 33])
torch.Size([1, 33])
tensor([[-0.0224-2.7048e-03j,  0.0062-9.1164e-03j, -0.0019+1.5549e-02j,
          0.0030+1.0849e-03j, -0.000

In [3]:
from lightning import LightningModule



class MyNet(LightningModule):
    def __init__(self, name, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        
        
a = MyNet('kk', bb=1)
a.hparams

"bb":   1
"name": kk